In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import xlrd

In [2]:
import pydoop.hdfs as hdfs
from pyhive import hive
from impala.dbapi import connect
from impala.util import as_pandas  
import pandas as pd
import numpy as np

In [4]:
# Establishing the connection with Hive
host_name = "10.107.6.14"
port = 10000
user = "hdfs"
password = "bigid#01"
database="referral_mart_latest"
conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
cursor_1 = conn.cursor()

**Data pre-processing**

Read all the csv files first.

In [3]:
df = pd.read_excel('Referral_VIN_Prime.xls')  #Prime VINs associated with taxi
taxi_vin_list = df.REFL_VIN.tolist()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
# Reading table refrl_vhcl_history_vin_final
cursor_1.execute('select * from refrl_vhcl_history_vin_final')
refrl_vhcl_history_vin = as_pandas(cursor_1)

In [7]:
refrl_vhcl_history_vin_final = refrl_vhcl_history_vin

In [8]:
print(refrl_vhcl_history_vin_final.shape)
print(refrl_vhcl_history_vin_final.columns)
refrl_vhcl_history_vin_final.head(3)

(25760815, 6)
Index(['vhcl_id', 'sale_dt', 'dlr_ou_id', 'repm_vin', 'vhcl_con_id',
       'phone_no'],
      dtype='object')


,vhcl_id,sale_dt,dlr_ou_id,repm_vin,vhcl_con_id,phone_no
0,1-2IJO-4542,2005-10-16,1-29J8-84,MALAA51HR5M700082,1-29OZ-1503,9891112121
1,1-2IJO-4919,2005-08-29,1-29J8-26,MALAA51HR5M700158,1-29OE-2971,2501201900
2,1-2ILN-4104,2006-07-14,1-29J8-300,MALAA51HR5M700178,1-2A9X-3345,9994155040


In [9]:
# Reading table cust_details
cursor_1.execute('select * from cust_details')
cust_details = as_pandas(cursor_1)

In [10]:
print(cust_details.shape)
print(cust_details.columns)
cust_details.head(3)

(4596173, 20)
Index(['cust_id', 'repm_dlr_no', 'repm_cust_no', 'repm_vin', 'm_mobl_phone_no',
       'con_id', 'person_uid', 'age', 'annl_incm_amt', 'birth_dt',
       'credit_score', 'job_title', 'occupation', 'sex_mf', 'marital_stat_cd',
       'state', 'title', 'con_type', 'contact_cell_phone',
       'master_cell_phone'],
      dtype='object')


,cust_id,repm_dlr_no,repm_cust_no,repm_vin,m_mobl_phone_no,con_id,person_uid,age,annl_incm_amt,birth_dt,credit_score,job_title,occupation,sex_mf,marital_stat_cd,state,title,con_type,contact_cell_phone,master_cell_phone
0,E1200B2015060270,E1200,B2015060270,MALAA51HR8M282087,9836211410,1-ET5S-26,HMI1007004716,None,None,None,None,null,null,M,null,CG,Mr.,I,9836211410,9836211410
1,E1200B2017060287,E1200,B2017060287,MALBM51BLGM246636,9830026536,1-IYDX-163,HMI1011605484,None,None,None,None,null,null,null,null,WB,null,I,9830026536,9830026536
2,E1200B2018110416,E1200,B2018110416,MALAA51HLCM778489,9830025617,1-RUNQ-1061,HMI1022736883,None,None,None,None,null,null,null,null,WB,null,I,9830025617,9830025617


In [11]:
#Remove used car VIN and merge service phone to referral VIN
# Reading table used_car_rcrepm
cursor_1.execute('select * from used_car_rcrepm')
used_car_rcrepm = as_pandas(cursor_1)

In [12]:
print(used_car_rcrepm.shape)
print(used_car_rcrepm.columns)
used_car_rcrepm.head(3)

(33744535, 14)
Index(['repm_dlr_code', 'repm_cust_no', 'repm_ro_no', 'repm_ro_dtime',
       'repm_vin', 'repm_work_type', 'repm_vst_type',
       'repm_bill_total_labr_amt', 'repm_bill_total_othr_amt',
       'repm_bill_total_part_amt', 'vinm_selling_date', 'vinm_sale_date',
       'vinm_model_year', 'vinm_last_milg'],
      dtype='object')


,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,null,0.00,0.00,0.00,null,20060629,2006,81635
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,null,0.00,0.00,0.00,null,20061128,2006,75950
2,W2213,C2008030086,R201702566,2017-03-13 18:08:27,MALCG51WR6M015479,OC,null,0.00,0.00,0.00,null,20061128,2006,75950


In [14]:
cust_details = cust_details.replace('null', np.nan)

In [15]:
used_car_rcrepm = used_car_rcrepm.replace('null',np.nan)

In [17]:
refrl_vhcl_history_vin = refrl_vhcl_history_vin.replace('null',np.nan)

In [18]:
vin_phone  = refrl_vhcl_history_vin_final.copy()
print(vin_phone.shape)
print(vin_phone.columns)
vin_phone.head(3)

(25760815, 6)
Index(['vhcl_id', 'sale_dt', 'dlr_ou_id', 'repm_vin', 'vhcl_con_id',
       'phone_no'],
      dtype='object')


,vhcl_id,sale_dt,dlr_ou_id,repm_vin,vhcl_con_id,phone_no
0,1-2IJO-4542,2005-10-16,1-29J8-84,MALAA51HR5M700082,1-29OZ-1503,9891112121
1,1-2IJO-4919,2005-08-29,1-29J8-26,MALAA51HR5M700158,1-29OE-2971,2501201900
2,1-2ILN-4104,2006-07-14,1-29J8-300,MALAA51HR5M700178,1-2A9X-3345,9994155040


In [19]:
service_phone  = cust_details.copy()
print(service_phone.shape)
print(service_phone.columns)
service_phone.head(3)

(4596173, 20)
Index(['cust_id', 'repm_dlr_no', 'repm_cust_no', 'repm_vin', 'm_mobl_phone_no',
       'con_id', 'person_uid', 'age', 'annl_incm_amt', 'birth_dt',
       'credit_score', 'job_title', 'occupation', 'sex_mf', 'marital_stat_cd',
       'state', 'title', 'con_type', 'contact_cell_phone',
       'master_cell_phone'],
      dtype='object')


,cust_id,repm_dlr_no,repm_cust_no,repm_vin,m_mobl_phone_no,con_id,person_uid,age,annl_incm_amt,birth_dt,credit_score,job_title,occupation,sex_mf,marital_stat_cd,state,title,con_type,contact_cell_phone,master_cell_phone
0,E1200B2015060270,E1200,B2015060270,MALAA51HR8M282087,9836211410,1-ET5S-26,HMI1007004716,None,None,None,None,NaN,NaN,M,NaN,CG,Mr.,I,9836211410,9836211410
1,E1200B2017060287,E1200,B2017060287,MALBM51BLGM246636,9830026536,1-IYDX-163,HMI1011605484,None,None,None,None,NaN,NaN,NaN,NaN,WB,NaN,I,9830026536,9830026536
2,E1200B2018110416,E1200,B2018110416,MALAA51HLCM778489,9830025617,1-RUNQ-1061,HMI1022736883,None,None,None,None,NaN,NaN,NaN,NaN,WB,NaN,I,9830025617,9830025617


In [20]:
# Remove Prime Taxi vins from both the above table
vin_phone = vin_phone[~vin_phone['repm_vin'].isin(taxi_vin_list)]
service_phone = service_phone[~service_phone['repm_vin'].isin(taxi_vin_list)]

In [21]:
#Phone number pre-processing
service_phone['Phone_no'] = np.where(((service_phone['master_cell_phone'].isnull())|
                            (service_phone['master_cell_phone'].astype(str).str.startswith('1'))|
                            (service_phone['master_cell_phone'].astype(str).str.startswith('0'))), 
                            service_phone['contact_cell_phone'], service_phone['master_cell_phone'])

#Remove suspicious phone numbers
service_phone_clean = service_phone[~service_phone['Phone_no'].astype(str).str.startswith('1')]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].astype(str).str.startswith('0')]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].astype(str).str.len()<10]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].isnull()]

In [22]:
print(service_phone_clean.shape)
print(service_phone_clean.columns)
service_phone_clean.head(3)

(4430815, 21)
Index(['cust_id', 'repm_dlr_no', 'repm_cust_no', 'repm_vin', 'm_mobl_phone_no',
       'con_id', 'person_uid', 'age', 'annl_incm_amt', 'birth_dt',
       'credit_score', 'job_title', 'occupation', 'sex_mf', 'marital_stat_cd',
       'state', 'title', 'con_type', 'contact_cell_phone', 'master_cell_phone',
       'Phone_no'],
      dtype='object')


,cust_id,repm_dlr_no,repm_cust_no,repm_vin,m_mobl_phone_no,con_id,person_uid,age,annl_incm_amt,birth_dt,...,job_title,occupation,sex_mf,marital_stat_cd,state,title,con_type,contact_cell_phone,master_cell_phone,Phone_no
0,E1200B2015060270,E1200,B2015060270,MALAA51HR8M282087,9836211410,1-ET5S-26,HMI1007004716,None,None,None,...,NaN,NaN,M,NaN,CG,Mr.,I,9836211410,9836211410,9836211410
1,E1200B2017060287,E1200,B2017060287,MALBM51BLGM246636,9830026536,1-IYDX-163,HMI1011605484,None,None,None,...,NaN,NaN,NaN,NaN,WB,NaN,I,9830026536,9830026536,9830026536
2,E1200B2018110416,E1200,B2018110416,MALAA51HLCM778489,9830025617,1-RUNQ-1061,HMI1022736883,None,None,None,...,NaN,NaN,NaN,NaN,WB,NaN,I,9830025617,9830025617,9830025617


In [23]:
#Additional list of suspicious service phone numbers
x = service_phone_clean.groupby(['master_cell_phone'])['repm_vin'].nunique().reset_index().sort_values(by=['repm_vin'], ascending=False)
Bad_service_phone = x[x['repm_vin']>50].master_cell_phone.tolist()
#Bad_service_phone list was further verified using True Caller API
#curl --request GET \
#  --url 'https://peoplestacks.com/api/v1/lookup/people/{phone_number}'
#  --header 'content-type: application/json'
len(Bad_service_phone)

30

**Create unique customer group id for non-used cars VIN**

In [69]:
used_car_serv = used_car_rcrepm.copy()

In [71]:
used_car_serv.dtypes

repm_dlr_code                       object
repm_cust_no                        object
repm_ro_no                          object
repm_ro_dtime               datetime64[ns]
repm_vin                            object
repm_work_type                      object
repm_vst_type                       object
repm_bill_total_labr_amt            object
repm_bill_total_othr_amt            object
repm_bill_total_part_amt            object
vinm_selling_date                   object
vinm_sale_date                      object
vinm_model_year                     object
vinm_last_milg                       int64
dtype: object

In [25]:
# Remove used car VIN and merge service phone to referral VIN
# used_car_serv = pd.read_csv('used_car_rcrepm.csv')
used_car_list = used_car_serv.repm_vin.tolist()
len(used_car_list)

33744535

In [26]:
# Reading table cars_with_used_tags
cursor_1.execute('select * from cars_with_used_tags')
car_with_used_tag = as_pandas(cursor_1)

In [27]:
car_with_used_tag = car_with_used_tag.replace('null', np.nan)

In [28]:
print(car_with_used_tag.shape)
print(car_with_used_tag.columns)
car_with_used_tag.head(3)

(1588120, 6)
Index(['vinm_vin', 'vinm_selling_date', 'repm_dlr_code', 'repm_dlr_no',
       'repm_cust_no', 'repm_ro_dtime'],
      dtype='object')


,vinm_vin,vinm_selling_date,repm_dlr_code,repm_dlr_no,repm_cust_no,repm_ro_dtime
0,MALCH41GRYM012111,20180501,N4212,N4212,C2016080175,2016-08-17 16:08:13
1,MALCH41GRYM012111,20180501,W4200,W4200,C2017051025,2017-05-31 18:17:01
2,MALCH41VR1M025259,20191114,S7202,S7202,C2019102145,2019-10-26 13:53:04


In [29]:
car_with_used_tag_list = car_with_used_tag.vinm_vin.tolist()
len(car_with_used_tag_list)

1588120

In [30]:
print("service_phone_clean", service_phone_clean.shape)
service_phone_notused = service_phone_clean[~service_phone_clean['repm_vin'].isin(car_with_used_tag_list)][[ 
    'repm_vin', 'con_id', 'Phone_no']].rename(columns={'Phone_no': 'service_phone'})
print(service_phone_notused.shape)
service_phone_notused.head(2)

service_phone_clean (4430815, 21)
(4176515, 3)


,repm_vin,con_id,service_phone
6,MALAA51HR3M358445,1-2BN1-1903,9836298621
7,MALAA51HR3M385680,1-2BUT-2847,9831077282


In [31]:
print("vin_phone", vin_phone.shape)
vin_phone_notused = vin_phone[~vin_phone['repm_vin'].isin(car_with_used_tag_list)][[
    'repm_vin', 'vhcl_con_id', 'phone_no']].rename(columns={'vhcl_con_id': 'con_id'})
print(vin_phone_notused.shape)
vin_phone_notused.head(2)

vin_phone (25728615, 6)
(24607162, 3)


,repm_vin,con_id,phone_no
0,MALAA51HR5M700082,1-29OZ-1503,9891112121
1,MALAA51HR5M700158,1-29OE-2971,2501201900


In [32]:
cust_notused = pd.merge(vin_phone_notused, service_phone_notused, how ='inner', on=['repm_vin'])

In [33]:
print(cust_notused.shape)
cust_notused.head(3)

(35407199, 5)


,repm_vin,con_id_x,phone_no,con_id_y,service_phone
0,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121
1,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121
2,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121


In [34]:
cust_notused['vin_grp'] = cust_notused.groupby(['repm_vin']).ngroup()+1
cust_notused.head(3)

,repm_vin,con_id_x,phone_no,con_id_y,service_phone,vin_grp
0,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852
1,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852
2,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852


In [35]:
cust_notused['ser_ph_grp'] = cust_notused.groupby(['service_phone']).ngroup()+1
cust_notused.head(3)

,repm_vin,con_id_x,phone_no,con_id_y,service_phone,vin_grp,ser_ph_grp
0,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486
1,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486
2,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486


In [36]:
phone_vin = cust_notused.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin = phone_vin[['ser_ph_grp', 'repm_vin']]
print(phone_vin.shape)
phone_vin.head()

(1682487, 2)


,ser_ph_grp,repm_vin
0,1,MALBM51BLJM504767
1,2,MALA851CLGM513092
2,3,MALBM51RLEM012428
3,4,MALSH81XLBM010678
4,5,MALA251ALJM602011


In [37]:
nwk = pd.merge(cust_notused[['ser_ph_grp', 'repm_vin']], phone_vin, how='left', on=['ser_ph_grp'])
print(nwk.shape)
nwk.head(2)

(63318479, 3)


,ser_ph_grp,repm_vin_x,repm_vin_y
0,1316486,MALAA51HR5M700082,MALAA51HR5M700082
1,1316486,MALAA51HR5M700082,MALAA51HR5M700082


In [38]:
#Need to run graph code with this var
nwk_dup_rm = nwk.drop_duplicates(subset = ['ser_ph_grp', 'repm_vin_x','repm_vin_y'], keep = 'last')
print(nwk_dup_rm.shape)
nwk_dup_rm.head(3)

(2825957, 3)


,ser_ph_grp,repm_vin_x,repm_vin_y
44,1316486,MALAA51HR5M700082,MALAA51HR5M700082
63,235,MALAA51HR5M700158,MALAA51HR5M700158
110,1567815,MALAA51HR5M700178,MALAA51HR5M700178


In [1]:
nwk_x = nwk.drop(['ser_ph_grp'], axis=1)

NameError: name 'nwk_dup_rm' is not defined

In [40]:
##Create network graph for service phone and VIN
G = nx.from_pandas_edgelist(nwk_x, source='repm_vin_x', target='repm_vin_y')
l = list(G.subgraph(x) for x in nx.connected_components(G))
d = dict((k, i) for i in range(len(l)) for k in l[i])
cust_notused['unique_cust_grp'] = cust_notused.repm_vin.map(d)
cust_notused = cust_notused[cust_notused.service_phone.notnull()]

In [41]:
print(cust_notused.shape)
cust_notused.head()

(35407199, 8)


,repm_vin,con_id_x,phone_no,con_id_y,service_phone,vin_grp,ser_ph_grp,unique_cust_grp
0,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486,0
1,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486,0
2,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486,0
3,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486,0
4,MALAA51HR5M700082,1-29OZ-1503,9891112121,1-29OZ-1503,9891112121,566852,1316486,0


**Create unique customer group id for used car VIN**

In [43]:
used_car_serv.head(2)

,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,NaN,0.00,0.00,0.00,NaN,20060629,2006,81635
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,NaN,0.00,0.00,0.00,NaN,20061128,2006,75950


In [102]:
used_car_serv.dtypes

repm_dlr_code                       object
repm_cust_no                        object
repm_ro_no                          object
repm_ro_dtime               datetime64[ns]
repm_vin                            object
repm_work_type                      object
repm_vst_type                       object
repm_bill_total_labr_amt            object
repm_bill_total_othr_amt            object
repm_bill_total_part_amt            object
vinm_selling_date           datetime64[ns]
vinm_sale_date                      object
vinm_model_year                     object
vinm_last_milg                       int64
used_car_sale_date_raw              object
used_car_sale_date          datetime64[ns]
dtype: object

In [45]:
used_car_serv['repm_ro_dtime'] = pd.to_datetime(used_car_serv['repm_ro_dtime'])

In [75]:
used_car_serv['vinm_selling_date'] = pd.to_datetime(used_car_serv['vinm_selling_date'])

In [76]:
used_car_serv["used_car_sale_date_raw"] = np.where((used_car_serv['vinm_selling_date'].notna()), used_car_serv['vinm_selling_date'].map(lambda x: str(x)[0:4]) + "-" + used_car_serv['vinm_selling_date'].map(lambda x: str(x)[5:7]) + "-" + used_car_serv['vinm_selling_date'].map(lambda x: str(x)[8:10]),used_car_serv['vinm_selling_date'])

In [ ]:
used_car_serv["used_car_sale_date_raw"] = used_car_serv["used_car_sale_date_raw"].replace(np.nan,"Test")
used_car_serv["used_car_sale_date_raw"] = used_car_serv["used_car_sale_date_raw"].replace("Test",np.nan)

In [100]:
used_car_serv['used_car_sale_date'] = pd.to_datetime(used_car_serv['used_car_sale_date_raw'].astype(str), format = '%Y-%m-%d')

In [101]:
used_car_serv.head(2)

,repm_dlr_code,repm_cust_no,repm_ro_no,repm_ro_dtime,repm_vin,repm_work_type,repm_vst_type,repm_bill_total_labr_amt,repm_bill_total_othr_amt,repm_bill_total_part_amt,vinm_selling_date,vinm_sale_date,vinm_model_year,vinm_last_milg,used_car_sale_date_raw,used_car_sale_date
0,N3208,C2008020258,R201708673,2017-07-29 10:10:50,MALCG51WR6M015343,OC,NaN,0.00,0.00,0.00,NaT,20060629,2006,81635,NaN,NaT
1,W2213,C2008030086,R201610998,2016-11-16 17:14:46,MALCG51WR6M015479,OC,NaN,0.00,0.00,0.00,NaT,20061128,2006,75950,NaN,NaT


In [103]:
used_car_serv_1 = used_car_serv[used_car_serv['repm_ro_dtime']< used_car_serv['used_car_sale_date']] #1st owner of the car

In [104]:
used_car_serv_2 = used_car_serv[used_car_serv['repm_ro_dtime']>=used_car_serv['used_car_sale_date']] #2nd owner of the car

In [105]:
print("used_car_serv whole dataset", used_car_serv.shape)
print("used_car_serv_1 :" , used_car_serv_1.shape)
print("used_car_serv_2 :" , used_car_serv_2.shape)

used_car_serv whole dataset (33744535, 16)
used_car_serv_1 : (922684, 16)
used_car_serv_2 : (649870, 16)


In [106]:
used_car_serv_2 = used_car_serv_2[used_car_serv_2['repm_work_type']!='PD'] #Remove service phone associated with 'PD' type
used_car_serv_1 = used_car_serv_1[used_car_serv_1['repm_work_type']!='PD'] #Remove service phone associated with 'PD' type

In [107]:
#Data pre-processing for used_car_serv_1. Remove bad phone numbers and merge to the vin_phone df. 
vin_phone_y = vin_phone[vin_phone['repm_vin'].isin(used_car_serv_1.repm_vin.unique().tolist())][
    ['repm_vin', 'vhcl_con_id', 'phone_no']].rename(columns={'vhcl_con_id': 'con_id'})

In [108]:
print(vin_phone_y.shape)
vin_phone_y.head(2)

(1009150, 3)


,repm_vin,con_id,phone_no
8,MALAA51HR5M700943,1-2B0Q-288,9685278310
9,MALAA51HR5M700944,1-2AU1-1483,9981290940


In [109]:
used_car_serv_1 = pd.merge(used_car_serv_1,service_phone_clean[['repm_vin','Phone_no']], on = 'repm_vin', how ='left', indicator = True)
used_car_serv_1['_merge'].value_counts()

both          1127854
left_only      223177
right_only          0
Name: _merge, dtype: int64

In [110]:
used_car_1_con = used_car_serv_1[~used_car_serv_1['Phone_no'].isin(Bad_service_phone)][['repm_vin', 'Phone_no']].rename(
columns={'Phone_no': 'service_phone'})
print(used_car_1_con.shape)
used_car_1_con.head(2)

(1347774, 2)


,repm_vin,service_phone
0,MALCH41GRYM012111,NaN
1,MALCH41GRYM012111,NaN


In [111]:
#used_car_1_con = used_car_1_con[used_car_1_con['service_phone'].notnull()]
used_car_1_con = used_car_1_con.drop_duplicates(subset=['service_phone'])

#used_car_1st = pd.merge(sales_vin_y, used_car_1_con, how='left', on=['repm_vin'])
used_car_1st = pd.merge(vin_phone_y, used_car_1_con, how='left', on=['repm_vin'])
used_car_1st = used_car_1st[used_car_1st.service_phone.notnull()]
print(used_car_1st.shape)
used_car_1st.head(3)

(1663924, 4)


,repm_vin,con_id,phone_no,service_phone
0,MALAA51HR5M700943,1-2B0Q-288,9685278310,9993666957
1,MALAA51HR5M700943,1-2B0Q-288,9685278310,9685278310
2,MALAA51HR5M700944,1-2AU1-1483,9981290940,9981290940


In [112]:
#Check how many service phone numbers present in used_car_serv_1 is present in cust_notused.
#These should be assigned same customer group as present in cust_notused
notused_dict = pd.Series(cust_notused['unique_cust_grp'].values, 
                        index=cust_notused['service_phone']).to_dict()

In [113]:
print("used_car_1st:", used_car_1st.shape)
used_car_1st_notused = used_car_1st[used_car_1st.service_phone.isin(cust_notused.service_phone.unique().tolist())]
print("used_car_1st_notused:", used_car_1st_notused.shape)

used_car_1st: (1663924, 4)
used_car_1st_notused: (262986, 4)


In [114]:
used_car_1st_notused = used_car_1st[used_car_1st.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_1st_notused.shape

(459599, 4)

In [115]:
used_car_1st_notused['unique_cust_grp'] = used_car_1st_notused['service_phone'].map(notused_dict)
print("used_car_1st_notused:", used_car_1st_notused.shape)
used_car_1st_notused.head(3)

used_car_1st_notused: (459599, 5)


/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,repm_vin,con_id,phone_no,service_phone,unique_cust_grp
14,MALAA51HR5M706952,1-29NP-4166,1234567890,9949241000,786935.0
29,MALAA51HR5M716803,1-29WV-2528,9412055310,9412055310,NaN
30,MALAA51HR5M716803,1-29WV-2528,9412055310,9675726092,385028.0


In [116]:
used_1_cust_dict = pd.Series(used_car_1st_notused['unique_cust_grp'].values, 
                        index=used_car_1st_notused['repm_vin']).to_dict()

In [117]:
used_car_1st_notused = used_car_1st[used_car_1st.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_1st_notused['unique_cust_grp'] = used_car_1st_notused['repm_vin'].map(used_1_cust_dict)    

used_car_1st_notused['vin_grp'] = 'used_car_1st'   #Identify customer groups with used cars. Such VIN will be present across multiple customer group
used_car_1st_notused['ser_ph_grp'] = 'same_notused' #Identify customer groups with used cars. 

used_car_1st_notused = used_car_1st_notused[['repm_vin', 'con_id', 'phone_no', 'service_phone', 'vin_grp',
                                             'ser_ph_grp', 'unique_cust_grp']]
cust_grp = pd.concat([cust_notused, used_car_1st_notused]) #Concat this to cust_notused

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [118]:
#Create unique customer group ids for used_car_1 that are not present in cust_notused
used_car_1st_unique = used_car_1st[~used_car_1st.repm_vin.isin(used_car_1st_notused.service_phone.unique().tolist())]
used_car_1st_unique['vin_grp'] = used_car_1st_unique.groupby(['repm_vin']).ngroup()+1
used_car_1st_unique['ser_ph_grp'] = used_car_1st_unique.groupby(['service_phone']).ngroup()+1

In [119]:
phone_vin_a = used_car_1st_unique.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin_a = phone_vin_a[['ser_ph_grp', 'repm_vin']]

nwk_used_1 = pd.merge(used_car_1st_unique[['ser_ph_grp', 'repm_vin']], phone_vin_a, how='left', on=['ser_ph_grp'])
nwk_used_1_x = nwk_used_1.drop(['ser_ph_grp'], axis=1)

In [120]:
G_used = nx.from_pandas_edgelist(nwk_used_1_x, source='repm_vin_x', target='repm_vin_y')
l = [list(G_used.subgraph(x)) for x in nx.connected_components(G_used)]
d_used = dict((k, i) for i in range(len(l)) for k in l[i])

In [121]:
used_car_1st_unique['unique_cust_grp_initial'] = used_car_1st_unique.repm_vin.map(d_used)

#Add a '_used' tag to cust_grp so that the group numbers are unique
used_car_1st_unique['unique_cust_grp'] = used_car_1st_unique['unique_cust_grp_initial'].astype(str) + '_' + 'used'
used_car_1st_unique = used_car_1st_unique.drop(['unique_cust_grp_initial'], axis=1)

#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_1st_unique])
cust_grp.shape

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(37530722, 9)

In [122]:
used_car_serv_2.shape

(649478, 16)

In [123]:
used_car_serv_2 = pd.merge(used_car_serv_2,service_phone_clean[['repm_vin', 'master_cell_phone', 'con_id','Phone_no']], on = 'repm_vin', how ='left',indicator=True)
used_car_serv_2['_merge'].value_counts()

both          899203
left_only     119648
right_only         0
Name: _merge, dtype: int64

In [124]:
used_car_serv_2.shape

(1018851, 20)

In [125]:
#Used_car_2 pre-processing
used_car_2nd = used_car_serv_2[used_car_serv_2.Phone_no.notnull()][['repm_vin', 'con_id', 'master_cell_phone',
                                                                    'Phone_no']].rename(
    columns={'master_cell_phone': 'phone_no', 'Phone_no':'service_phone'})
used_car_2nd = used_car_2nd[~used_car_2nd.service_phone.isin(Bad_service_phone)]

In [126]:
#Check how many service phone numbers present in used_car_serv_2 is present in cust_grp.
#These should be assigned same customer group as present in cust_grp
all_dict = pd.Series(cust_grp['unique_cust_grp'].values, 
                        index=cust_grp['service_phone']).to_dict()

used_car_2nd_notused = used_car_2nd[used_car_2nd.service_phone.isin(cust_grp.service_phone.unique().tolist())]
used_car_2nd_notused = used_car_2nd[used_car_2nd.repm_vin.isin(used_car_2nd_notused.repm_vin.unique().tolist())]
used_car_2nd_notused['unique_cust_grp'] = used_car_2nd_notused['service_phone'].map(all_dict)
used_2_cust_dict = pd.Series(used_car_2nd_notused['unique_cust_grp'].values, 
                        index=used_car_2nd_notused['repm_vin']).to_dict()
used_car_2nd_notused = used_car_2nd[used_car_2nd.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_2nd_notused['unique_cust_grp'] = used_car_2nd_notused['repm_vin'].map(used_1_cust_dict)    

used_car_2nd_notused['vin_grp'] = 'used_car_2nd'    #Identify customer groups with used cars.
used_car_2nd_notused['ser_ph_grp'] = 'same_notused'  #Identify customer groups with used cars.

used_car_2nd_notused = used_car_2nd_notused[['repm_vin', 'con_id', 'phone_no', 'service_phone', 'vin_grp',
                                             'ser_ph_grp', 'unique_cust_grp']]

#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_2nd_notused])

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [127]:
cust_grp.shape

(37585221, 9)

In [128]:
#Create unique customer group ids for used_car_2 that are not present in cust_grp

used_car_2nd_unique = used_car_2nd[~used_car_2nd.repm_vin.isin(used_car_2nd_notused.repm_vin.unique().tolist())]
used_car_2nd_unique = used_car_2nd_unique.drop_duplicates(subset=['repm_vin', 'con_id', 'service_phone'])
used_car_2nd_unique['vin_grp'] = used_car_2nd_unique.groupby(['repm_vin']).ngroup()+1
used_car_2nd_unique['ser_ph_grp'] = used_car_2nd_unique.groupby(['service_phone']).ngroup()+1
phone_vin_b = used_car_2nd_unique.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin_b = phone_vin_b[['ser_ph_grp', 'repm_vin']]
nwk_used_2 = pd.merge(used_car_2nd_unique[['ser_ph_grp', 'repm_vin']], phone_vin_b, how='left', on=['ser_ph_grp'])
nwk_used_2_x = nwk_used_2.drop(['ser_ph_grp'], axis=1)

In [129]:
# Create the graph
G_used_2 = nx.from_pandas_edgelist(nwk_used_2_x, source='repm_vin_x', target='repm_vin_y')
l = [list(G_used_2.subgraph(x)) for x in nx.connected_components(G_used_2)]
d_used_2 = dict((k, i) for i in range(len(l)) for k in l[i])

In [130]:
used_car_2nd_unique['unique_cust_grp_initial'] = used_car_2nd_unique.repm_vin.map(d_used_2)

#Add a '_used2' tag to cust_grp so that the group numbers are unique
used_car_2nd_unique['unique_cust_grp'] = used_car_2nd_unique['unique_cust_grp_initial'].astype(str) + '_' + 'used2'
used_car_2nd_unique = used_car_2nd_unique.drop(['unique_cust_grp_initial'], axis=1)
#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_2nd_unique])
cust_grp.shape

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


(37776982, 9)

In [131]:
#Remove duplicates
cust_grp_final = cust_grp.drop_duplicates(subset=['repm_vin', 'con_id', 'phone_no', 'service_phone'])
cust_grp_final.shape

(1924867, 9)

In [146]:
#cust_grp.reset_index().to_excel("Cust_grp_master.xlsx")
#cust_grp_final.reset_index().to_excel("Cust_grp_master_unique.xlsx")
cust_grp.reset_index().to_json("Cust_grp_master.JSON")

In [139]:
# Add Target column to cust_grp_final df to identify customer group that provided referral vs those that did not

# refrl_tag = pd.read_excel('crm_rcrefl_tb_complete.xlsx') #Referral master data
cursor_1.execute('select * from crm_rcrefl_tb')
refrl_tag = as_pandas(cursor_1)

In [140]:
rfrl_data = refrl_tag[refrl_tag['refl_vin'].astype(str).str.startswith('MA')|
         refrl_tag['refl_vin'].astype(str).str.startswith('KM')] # Only Hyundai VIN


In [143]:
cust_grp_final['Target'] = np.where(cust_grp_final['repm_vin'].isin(rfrl_data['refl_vin'].unique().tolist()),
                                   1, 0)

#Identify all unique_cust_grp containing VINs that provided referrals and map it to Target colum
unique_1 = cust_grp_final[cust_grp_final['Target']==1].unique_cust_grp.unique().tolist()

cust_grp_final['Target'] = np.where(cust_grp_final['unique_cust_grp'].isin(unique_1),
                                   1, 0)
cust_grp_final.shape

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(1924867, 10)

In [144]:
cust_grp_final.head(5)

,con_id,con_id_x,con_id_y,phone_no,repm_vin,ser_ph_grp,service_phone,unique_cust_grp,vin_grp,Target
0,NaN,1-29OZ-1503,1-29OZ-1503,9891112121,MALAA51HR5M700082,1316486,9891112121,0,566852,0
45,NaN,1-29OE-2971,1-29OE-2971,2501201900,MALAA51HR5M700158,235,2501201900,1,566853,0
64,NaN,1-2A9X-3345,1-2A9X-3345,9994155040,MALAA51HR5M700178,1567815,9994155040,2,566854,0
111,NaN,1-2B7U-3041,1-2B7U-3041,9880977808,MALAA51HR5M700666,672422,9448346265,3,566855,0
112,NaN,1-2B7U-3041,1-F817-67,9880977808,MALAA51HR5M700666,1284729,9880977808,3,566855,0


In [147]:
#Save the above df and use for feature_engineering.ipynb
cust_grp_final.to_csv('Cust_grp_master_final_1.csv', encoding = 'utf-8', index=False)

print("Done")

Done
